# model for recognition of shirts

In [1]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import numpy as np

# os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

from tensorflow.keras.preprocessing.image import ImageDataGenerator


2024-12-11 14:28:34.721341: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 14:28:34.721750: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-11 14:28:34.724827: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-11 14:28:34.731992: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733907514.743499   23754 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733907514.74

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/edunet-01/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/edunet-01/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/edunet-01/.local/lib/python3.10

AttributeError: _ARRAY_API not found

In [3]:
!pip install keras

Defaulting to user installation because normal site-packages is not writeable


In [4]:
pip install --upgrade scipy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [13]:


# Directory paths
train_dir = 'dataset/train/upperwear'
test_dir = 'dataset/test/upperwear'

# Preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=30,  # Random rotation
    width_shift_range=0.2,  # Horizontal shift
    height_shift_range=0.2,  # Vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Zoom
    horizontal_flip=True,  # Flip images
    fill_mode='nearest'  # Fill in missing pixels
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load training and testing data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize all images to 150x150
    batch_size=32,
    class_mode='categorical'  # Multi-class classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 9347 images belonging to 3 classes.
Found 1181 images belonging to 3 classes.


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
model = Sequential([
    # Convolutional Layer 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    # Convolutional Layer 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Convolutional Layer 3
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flattening the feature map
    Flatten(),

    # Fully Connected Layer
    Dense(128, activation='relu'),
    Dropout(0.5),  # Regularization to prevent overfitting

    # Output Layer
    Dense(3, activation='softmax')  # 4 categories: solid, checks, stripes, patterns
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,739 (18.42 MB)

 Trainable params: 4,828,739 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)


Epoch 1/20
292/292 ━━━━━━━━━━━━━━━━━━━━ 131s 450ms/step - accuracy: 0.7476 - loss: 0.6111 - val_accuracy: 0.8151 - val_loss: 0.5347
Epoch 2/20
  1/292 ━━━━━━━━━━━━━━━━━━━━ 1:13 251ms/step - accuracy: 0.8125 - loss: 0.6029

/home/edunet-01/.local/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


292/292 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.8125 - loss: 0.6029 - val_accuracy: 0.7795 - val_loss: 0.5586
Epoch 3/20
292/292 ━━━━━━━━━━━━━━━━━━━━ 127s 434ms/step - accuracy: 0.7488 - loss: 0.6160 - val_accuracy: 0.8220 - val_loss: 0.4764
Epoch 4/20
292/292 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.8125 - loss: 0.5427 - val_accuracy: 0.8255 - val_loss: 0.4784
Epoch 5/20
292/292 ━━━━━━━━━━━━━━━━━━━━ 136s 466ms/step - accuracy: 0.7638 - loss: 0.5682 - val_accuracy: 0.8238 - val_loss: 0.4549
Epoch 6/20
292/292 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.6875 - loss: 0.6686 - val_accuracy: 0.8247 - val_loss: 0.4497
Epoch 7/20
292/292 ━━━━━━━━━━━━━━━━━━━━ 136s 466ms/step - accuracy: 0.7747 - loss: 0.5439 - val_accuracy: 0.8212 - val_loss: 0.4556
Epoch 8/20
292/292 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.7188 - loss: 0.5569 - val_accuracy: 0.8212 - val_loss: 0.4576
Epoch 9/20
292/292 ━━━━━━━━━━━━━━━━━━━━ 132s 451ms/step - accuracy: 0.7902 - loss: 0.5137 - val_acc

In [35]:
from tensorflow.keras.preprocessing import image

# Load and preprocess a single image
img = image.load_img('tshirt.jpeg', target_size=(150, 150))
img_array = image.img_to_array(img) / 255.0  # Normalize the image
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict the class
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)
class_labels = list(train_generator.class_indices.keys())
print(f"Predicted Class: {class_labels[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted Class: tshirt


In [39]:
# Directory paths
train_dir_bottom = 'dataset/train/bottomwear'
test_dir_bottom = 'dataset/test/bottomwear'

# Preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=30,  # Random rotation
    width_shift_range=0.2,  # Horizontal shift
    height_shift_range=0.2,  # Vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Zoom
    horizontal_flip=True,  # Flip images
    fill_mode='nearest'  # Fill in missing pixels
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load training and testing data
train_generator = train_datagen.flow_from_directory(
    train_dir_bottom ,
    target_size=(150, 150),  # Resize all images to 150x150
    batch_size=32,
    class_mode='categorical'  # Multi-class classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir_bottom ,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 10582 images belonging to 3 classes.
Found 1313 images belonging to 3 classes.


In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
model_bottom = Sequential([
    # Convolutional Layer 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    # Convolutional Layer 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Convolutional Layer 3
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flattening the feature map
    Flatten(),

    # Fully Connected Layer
    Dense(128, activation='relu'),
    Dropout(0.5),  # Regularization to prevent overfitting

    # Output Layer
    Dense(3, activation='softmax')  # 4 categories: solid, checks, stripes, patterns
])

# Compile the model
model_bottom.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model_bottom.summary()


/home/edunet-01/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,739 (18.42 MB)

 Trainable params: 4,828,739 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
history = model_bottom.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

Epoch 1/20
330/330 ━━━━━━━━━━━━━━━━━━━━ 150s 452ms/step - accuracy: 0.5691 - loss: 0.8945 - val_accuracy: 0.8178 - val_loss: 0.4513
Epoch 2/20
330/330 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.6562 - loss: 0.6100 - val_accuracy: 0.8255 - val_loss: 0.4427
Epoch 3/20
330/330 ━━━━━━━━━━━━━━━━━━━━ 147s 444ms/step - accuracy: 0.7453 - loss: 0.5650 - val_accuracy: 0.8651 - val_loss: 0.4061
Epoch 4/20
330/330 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.8750 - loss: 0.4794 - val_accuracy: 0.8544 - val_loss: 0.4163
Epoch 5/20
330/330 ━━━━━━━━━━━━━━━━━━━━ 148s 448ms/step - accuracy: 0.7850 - loss: 0.4816 - val_accuracy: 0.8902 - val_loss: 0.3029
Epoch 6/20
330/330 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.7812 - loss: 0.5972 - val_accuracy: 0.9032 - val_loss: 0.2979
Epoch 7/20
330/330 ━━━━━━━━━━━━━━━━━━━━ 149s 450ms/step - accuracy: 0.8248 - loss: 0.4243 - val_accuracy: 0.8918 - val_loss: 0.3418
Epoch 8/20
330/330 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.8125 - loss: 0.421

In [44]:
from tensorflow.keras.preprocessing import image

# Load and preprocess a single image
img = image.load_img('one-piece.jpeg', target_size=(150, 150))
img_array = image.img_to_array(img) / 255.0  # Normalize the image
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict the class
predictions = model_bottom.predict(img_array)
predicted_class = np.argmax(predictions)
class_labels = list(train_generator.class_indices.keys())
print(f"Predicted Class: {class_labels[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Predicted Class: pants
